In [1]:
import numpy as np
import pandas as pd
from account import Account
from brokerage import Brokerage
from datastream import *

# Account Class

In [2]:
accounts = {'linsu_binance': Account(10000, positions={'btc': [1, 65000], 'eth':[3, 4300]}),
            'linsu_coinbase': Account(10000, positions={'bnb': [10, 500], 'eth':[-1, 4700]})}

accounts['linsu_binance'].summary()
print('---')
accounts['linsu_coinbase'].summary()

Cash Balance: 10000
Asset Balance: 77900
Total Balance: 87900
Positions: {'btc': [1, 65000], 'eth': [3, 4300]}
---
Cash Balance: 10000
Asset Balance: 300
Total Balance: 10300
Positions: {'bnb': [10, 500], 'eth': [-1, 4700]}


In [3]:
# updates current price of BTC
accounts['linsu_binance'].update_position('btc', 0, 65000)

accounts['linsu_binance'].summary()
print('---')
accounts['linsu_coinbase'].summary()

Cash Balance: 10000
Asset Balance: 77900
Total Balance: 87900
Positions: {'btc': [1, 65000], 'eth': [3, 4300]}
---
Cash Balance: 10000
Asset Balance: 300
Total Balance: 10300
Positions: {'bnb': [10, 500], 'eth': [-1, 4700]}


In [4]:
# buys 1 BTC at 45000; simultaneously updates current price of BTC
accounts['linsu_binance'].update_position('btc', 1, 45000)

accounts['linsu_binance'].summary()
print('---')
accounts['linsu_coinbase'].summary()

Cash Balance: -35000
Asset Balance: 102900
Total Balance: 67900
Positions: {'btc': [2, 45000], 'eth': [3, 4300]}
---
Cash Balance: 10000
Asset Balance: 300
Total Balance: 10300
Positions: {'bnb': [10, 500], 'eth': [-1, 4700]}


In [5]:
# shorts 10000 CRO at .45; simultaneously updates current price of CRO
accounts['linsu_coinbase'].update_position('cro', -10000, .45)

accounts['linsu_binance'].summary()
print('---')
accounts['linsu_coinbase'].summary()

Cash Balance: -35000
Asset Balance: 102900
Total Balance: 67900
Positions: {'btc': [2, 45000], 'eth': [3, 4300]}
---
Cash Balance: 14500.0
Asset Balance: -4200.0
Total Balance: 10300.0
Positions: {'bnb': [10, 500], 'eth': [-1, 4700], 'cro': [-10000, 0.45]}


# Data

In [6]:
# Random walk of Feb 12 data
dict_data = {'btc': np.random.normal(42728, 42843*.01, 100),
             'eth': np.random.normal(2957.3, 2957.3*.01, 100),
             'bnb': np.random.normal(403.67, 403.67*.01, 100),
             'neo': np.random.normal(21.909, 21.909*.01, 100)}

In [7]:
df_data = pd.DataFrame(dict_data)
df_data

,btc,eth,bnb,neo
0,43257.079038,2950.352082,397.515385,21.817841
1,42677.827774,2969.647831,402.913092,22.188995
2,42737.543310,2965.862386,402.811412,22.030002
3,42860.310838,3017.988188,397.446024,22.114893
4,42746.641468,2972.507399,399.147442,21.474041
...,...,...,...,...
95,43353.655431,2969.880019,400.767402,21.607636
96,42677.233283,2999.547714,408.015769,22.349903
97,42829.272048,2980.054302,399.512979,22.158007
98,42523.765086,3001.035268,409.139964,21.880429


# Brokerage Class

In [8]:
datastream = PandasDataStream(df_data)
accounts = {'linsu_binance': Account(10000, positions={'btc': [1, 65000], 'eth':[3, 4300]}),
            'linsu_coinbase': Account(10000, positions={'bnb': [10, 500], 'eth':[-1, 4700]})}
b = Brokerage(datastream, accounts)

b.accounts['linsu_binance'].summary()
print('---')
b.accounts['linsu_coinbase'].summary()

Cash Balance: 10000
Asset Balance: 77900
Total Balance: 87900
Positions: {'btc': [1, 65000], 'eth': [3, 4300]}
---
Cash Balance: 10000
Asset Balance: 300
Total Balance: 10300
Positions: {'bnb': [10, 500], 'eth': [-1, 4700]}


In [9]:
# next updates the prices of all assets in account and processes any orders if there exists any
next(b)

b.accounts['linsu_binance'].summary()
print('---')
b.accounts['linsu_coinbase'].summary()

Cash Balance: 10000.0
Asset Balance: 52108.13528545539
Total Balance: 62108.13528545539
Positions: {'btc': [1, 43257.079038353186], 'eth': [3, 2950.352082367402]}
---
Cash Balance: 10000.0
Asset Balance: 1024.801769052809
Total Balance: 11024.801769052809
Positions: {'bnb': [10, 397.5153851420211], 'eth': [-1, 2950.352082367402]}


In [10]:
# market buy
b.add_order('linsu_binance', 'neo', '9fd8s0', 'market', -1)
next(b)

b.accounts['linsu_binance'].summary()
print('---')
b.accounts['linsu_coinbase'].summary()

Cash Balance: 10022.188994945689
Asset Balance: 51564.5822705489
Total Balance: 61586.771265494586
Positions: {'btc': [1, 42677.827773942976], 'eth': [3, 2969.647830517203], 'neo': [-1, 22.188994945688922]}
---
Cash Balance: 10000.0
Asset Balance: 1059.483092399642
Total Balance: 11059.483092399641
Positions: {'bnb': [10, 402.9130922916845], 'eth': [-1, 2969.647830517203]}


In [11]:
# limit buy triggers since bnb price < limit buy price
b.add_order('linsu_binance', 'bnb', '9fd8s0fdsa', 'limit', 5, 30)
next(b)

b.accounts['linsu_binance'].summary()
print('---')
b.accounts['linsu_coinbase'].summary()

Cash Balance: 10022.188994945689
Asset Balance: 51613.100468080884
Total Balance: 61635.28946302657
Positions: {'btc': [1, 42737.543310443725], 'eth': [3, 2965.862386456563], 'neo': [-1, 22.0300017325274]}
---
Cash Balance: 10000.0
Asset Balance: 1062.2517319708418
Total Balance: 11062.251731970842
Positions: {'bnb': [10, 402.8114118427405], 'eth': [-1, 2965.862386456563]}


In [12]:
# limit buy fails to trigger since bnb price > limit buy price
b.add_order('linsu_binance', 'bnb', '9fd8s0fdsa', 'limit', 5, 20)
next(b)

b.accounts['linsu_binance'].summary()
print('---')
b.accounts['linsu_coinbase'].summary()

Cash Balance: 10022.188994945689
Asset Balance: 51892.160508713154
Total Balance: 61914.34950365884
Positions: {'btc': [1, 42860.31083791815], 'eth': [3, 3017.988187886029], 'neo': [-1, 22.114892863087245]}
---
Cash Balance: 10000.0
Asset Balance: 956.4720528270318
Total Balance: 10956.472052827032
Positions: {'bnb': [10, 397.4460240713061], 'eth': [-1, 3017.988187886029]}
